In [84]:
# !pip install geopandas scipy geopy --quiet

In [85]:
import itertools
from operator import itemgetter

import geopandas as gpd
import numpy as np
import pandas as pd

from scipy.spatial import cKDTree
from tqdm.notebook import tqdm

import matplotlib 
import geopy.distance
import matplotlib.pyplot as plt
from math import sqrt
SPINE_COLOR = 'gray'
# %matplotlib inline

In [86]:
def ckdnearest(brick_kilns, rivers, gdfB_cols=['geometry']):
    A = np.vstack([np.array(geom) for geom in brick_kilns[['lon','lat']].values])
    # print(A)
    B = [np.array(geom.coords) for geom in rivers.geometry.to_list()] 
    # print(B)
    B_ix = tuple(itertools.chain.from_iterable(
        [itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))]))
    # print(B_ix)
    B = np.concatenate(B)
    # B = np.delete(B, 2, axis = 1) #removing z (altitude) component 
    # print(B)
    ckd_tree = cKDTree(B)
    dist, river_point_idx = ckd_tree.query(A, k=1)
    # print(dist)
    # print(river_point_idx)
    closest_river_point = B[river_point_idx]
    # print(B[river_point_idx])
    river_origin_idx = itemgetter(*river_point_idx)(B_ix)
    gdf = pd.concat(
        [brick_kilns, rivers.loc[river_origin_idx, gdfB_cols].reset_index(drop=True),
         pd.DataFrame(closest_river_point, columns = ['closest_river_point_long','closest_river_point_lat']),
         pd.Series(dist, name='dist')], axis=1)
    return gdf

def cal_bk_hosp_dist(path):
    state_bk =pd.read_csv(path)
    bk_hospital_mapping = ckdnearest_hospital(state_bk, hospital_df)
    # print(bk_hospital_mapping)
    bk_hospital_mapping['distance_km'] = 0

    for i in range(len(bk_hospital_mapping)):
        bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km

    return bk_hospital_mapping

In [87]:
#https://mapcruzin.com/free-india-country-city-place-gis-shapefiles.htm#google_vignette

waterways_path = '/home/shataxi.dubey/shataxi_work/compliance_analysis/waterways/waterways.shp'
waterways = gpd.read_file(waterways_path)
# waterways[waterways['name'] == 'Ganga']['geometry']

In [88]:
fp2 = "/home/shataxi.dubey/shataxi_work/India_State_Shapefile/India_State_Shapefile/India_State_Boundary.shp"

# Read file using gpd.read_file()
data2 = gpd.read_file(fp2)

In [89]:
def cal_bk_river_dist(path):
    state_bk = pd.read_csv(path)
    # print(state_bk.columns )
    # state_bk.columns = ['lat', 'lon']
    # print(state_bk.columns )
    bk_river_mapping = ckdnearest(state_bk, waterways)
    bk_river_mapping['distance'] = 0
    # bk_river_mapping

    for i in range(len(state_bk)):
        bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km

    return bk_river_mapping

In [90]:
def latexify(fig_width=None, fig_height=None, columns=1):
    """Set up matplotlib's RC params for LaTeX plotting.
    Call this before plotting a figure.

    Parameters
    ----------
    fig_width : float, optional, inches
    fig_height : float,  optional, inches
    columns : {1, 2}
    """

    # code adapted from http://www.scipy.org/Cookbook/Matplotlib/LaTeX_Examples

    # Width and max height in inches for IEEE journals taken from
    # computer.org/cms/Computer.org/Journal%20templates/transactions_art_guide.pdf

    assert(columns in [1,2])

    if fig_width is None:
        fig_width = 3.39 if columns==1 else 6.9 # width in inches

    if fig_height is None:
        golden_mean = (sqrt(5)-1.0)/2.0    # Aesthetic ratio
        fig_height = fig_width*golden_mean # height in inches

    MAX_HEIGHT_INCHES = 8.0
    if fig_height > MAX_HEIGHT_INCHES:
        print("WARNING: fig_height too large:" + fig_height + 
              "so will reduce to" + MAX_HEIGHT_INCHES + "inches.")
        fig_height = MAX_HEIGHT_INCHES

    params = {'backend': 'ps',
#              'text.latex.preamble': [r'\usepackage{gensymb}'],
              'axes.labelsize': 8, # fontsize for x and y labels (was 10)
              'axes.titlesize': 8,
              'font.size': 8, # was 10
              'legend.fontsize': 8, # was 10
              'xtick.labelsize': 8,
              'ytick.labelsize': 8,
              'text.usetex': True,
              'figure.figsize': [fig_width,fig_height],
              'font.family': 'serif'
    }

    matplotlib.rcParams.update(params)


def format_axes(ax):

    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)

    for spine in ['left', 'bottom']:
        ax.spines[spine].set_color(SPINE_COLOR)
        ax.spines[spine].set_linewidth(0.5)

    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_tick_params(direction='out', color=SPINE_COLOR)

    return ax

In [91]:
# fig_width_pt = 241.14749  # Get this from LaTeX using \the\columnwidth
fig_width_pt = 450
inches_per_pt = 1.0/72.27               # Convert pt to inches
# golden_mean = (sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
# fig_height =fig_width*golden_mean       # height in inches
# fig_height =fig_width
fig_height = 3
fig_size = [fig_width,fig_height]
fig_size

[6.226650062266501, 3]

## Get all violations

In [98]:
states = ["punjab", "haryana", "bihar", "jharkhand", "madhya_pradesh", "west_bengal", "uttar_pradesh", "himachal_pradesh", "uttarakhand"]

river_df = pd.DataFrame()
for state in tqdm(states):
    mapping = cal_bk_river_dist(f"/home/patel_zeel/compass24/exact_latlon/{state}.csv")
    river_df.loc[state, "total"] = len(mapping)
    river_df.loc[state, "violations"] = len(mapping[mapping['distance'] <= 0.5])

  0%|          | 0/9 [00:00<?, ?it/s]

/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km
/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km
/tmp/ipykernel_21368

In [99]:
river_df.loc["Total"] = river_df.sum()
river_df.loc["Total", "violations"] / river_df.loc["Total", "total"]

0.09014760713008836

In [100]:
river_df

,total,violations
punjab,2327.0,182.0
haryana,2351.0,416.0
bihar,4138.0,299.0
jharkhand,366.0,104.0
madhya_pradesh,212.0,33.0
west_bengal,2291.0,383.0
uttar_pradesh,7510.0,299.0
himachal_pradesh,151.0,25.0
uttarakhand,233.0,24.0
Total,19579.0,1765.0


In [95]:
hospital_res = pd.DataFrame()
hospital_df = pd.read_csv('/home/rishabh.mondal/bkdb/India_Hospital_Data.csv')
hospital_df = hospital_df.rename(columns = {'lon' : 'lon', 'lat' : 'lat'})

for state in tqdm(states):
    mapping = cal_bk_hosp_dist(f'/home/patel_zeel/compass24/exact_latlon/{state}.csv')
    hospital_res.loc[state, "total"] = len(mapping)
    hospital_res.loc[state, "violations"] = len(mapping[mapping['distance_km'] <= 0.8])

  0%|          | 0/9 [00:00<?, ?it/s]

/tmp/ipykernel_2136890/3911981859.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km
/tmp/ipykernel_2136890/3911981859.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i]))

In [96]:
hospital_res.loc["Total"] = hospital_res.sum()
hospital_res.loc["Total", "violations"] / hospital_res.loc["Total", "total"]

0.1561366770519434

In [97]:
hospital_res

,total,violations
punjab,2327.0,253.0
haryana,2351.0,93.0
bihar,4138.0,409.0
jharkhand,366.0,35.0
madhya_pradesh,212.0,15.0
west_bengal,2291.0,584.0
uttar_pradesh,7510.0,1609.0
himachal_pradesh,151.0,37.0
uttarakhand,233.0,22.0
Total,19579.0,3057.0


## Combined violations

In [105]:
combined_res = pd.DataFrame()

for state in tqdm(states):
    hosp_mapping = cal_bk_hosp_dist(f'/home/patel_zeel/compass24/exact_latlon/{state}.csv')
    river_mapping = cal_bk_river_dist(f"/home/patel_zeel/compass24/exact_latlon/{state}.csv")
    
    combined_res.loc[state, "total"] = len(hosp_mapping)
    combined_res.loc[state, "violations"] = len(hosp_mapping[(hosp_mapping['distance_km'] <= 0.8) & (river_mapping['distance'] <= 0.5)])
    
combined_res.loc["Total"] = combined_res.sum()
combined_res.loc["Total", "violations"] / combined_res.loc["Total", "total"]

  0%|          | 0/9 [00:00<?, ?it/s]

/tmp/ipykernel_2136890/3911981859.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km
/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km
/tmp/ipy

0.013790285509985189

In [106]:
combined_res

,total,violations
punjab,2327.0,30.0
haryana,2351.0,20.0
bihar,4138.0,30.0
jharkhand,366.0,5.0
madhya_pradesh,212.0,6.0
west_bengal,2291.0,98.0
uttar_pradesh,7510.0,66.0
himachal_pradesh,151.0,10.0
uttarakhand,233.0,5.0
Total,19579.0,270.0


## JHARKHAND

In [65]:
bk_river_mapping = cal_bk_river_dist('/home/patel_zeel/compass24/exact_latlon/jharkhand.csv')
bk_river_mapping.head(2)

/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km


,parent,line_id,class,lat,lon,state,x_c,y_c,w,h,...,bottom_right_y,local_top_left_x,local_top_left_y,local_bottom_right_x,local_bottom_right_y,geometry,closest_river_point_long,closest_river_point_lat,dist,distance
0,"24.10,83.72",0,fcbk,24.103510,83.717100,jharkhand,2.458022e+07,1.446125e+07,0.038356,0.060851,...,1.446128e+07,268.204328,167.502328,311.162712,235.655112,"LINESTRING (83.55034 24.11625, 83.55208 24.116...",83.713891,24.124086,0.020825,2.302204
1,"23.52,84.88",0,fcbk,23.515956,84.881901,jharkhand,2.468878e+07,1.452111e+07,0.031068,0.066962,...,1.452114e+07,719.816048,933.605232,754.612432,1008.602448,"LINESTRING (85.13301 23.41640, 85.13288 23.416...",84.868648,23.469024,0.048767,5.371149


In [66]:
latexify(fig_width, fig_height)


In [67]:
fig, ax = plt.subplots()

waterways.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'blue', linewidth = 0.2, label = 'rivers', alpha=0.8) # river data
data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.2) # UP map

# bk - river distance > 0.5 KM
dist_1 = bk_river_mapping[(bk_river_mapping['distance'] > 0.5)]
ax.scatter(dist_1['lon'],dist_1['lat'], color='green', s=0.5, marker = 'o', linewidth = 0.5, label = f'{len(dist_1)} brick kilns $>$ 0.5 Km')

# bk - river distance < 0.5 KM
dist_1 = bk_river_mapping[bk_river_mapping['distance'] <= 0.5]
ax.scatter(dist_1['lon'],dist_1['lat'], color='red', s=2, marker = 'o',linewidth = 0.5, label = f'{len(dist_1)} brick kilns $<$ 0.5 Km')

# ax.text(76.1, 31.8, 'Himachal\n Pradesh')
ax.text(85.1, 24.93, 'Bihar')
# ax.text(73.8, 26.23, 'Rajasthan')
ax.text(84.6, 23.45, 'Jharkhand')
ax.text(86.9, 23.05, 'West\n Bengal')
# ax.text(75.2, 21.0, 'Maharashtra')
ax.text(83.05, 22.9, 'Chhattisgarh')

ax.set_xlim(82.97,88.00)
ax.set_ylim(21.87,25.63)
plt.legend( loc='upper left', borderaxespad=0.)

ax.set_axis_off()
format_axes(ax)
# plt.savefig('punjab_bk_river.pdf')
plt.tight_layout()
# plt.savefig('jharkhand_bk_river.pdf')
# plt.savefig('jharkhand_bk_river.png', dpi=300)

## WEST BENGAL

In [14]:
bk_river_mapping = cal_bk_river_dist('/home/patel_zeel/compass24/exact_latlon/west_bengal.csv')
bk_river_mapping.head(2)

/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km


,parent,line_id,class,lat,lon,state,x_c,y_c,w,h,...,bottom_right_y,local_top_left_x,local_top_left_y,local_bottom_right_x,local_bottom_right_y,geometry,closest_river_point_long,closest_river_point_lat,dist,distance
0,"22.25,88.55",0,zigzag,22.249829,88.554479,west_bengal,2.503109e+07,1.464920e+07,0.044787,0.069990,...,1.464924e+07,952.436128,538.016808,1002.597792,616.405272,"LINESTRING (88.51401 22.34519, 88.51510 22.345...",88.604777,22.300813,0.071618,7.664456
1,"23.73,87.71",0,zigzag,23.732203,87.715413,west_bengal,2.495289e+07,1.449911e+07,0.065385,0.042536,...,1.449913e+07,1027.919648,311.870832,1101.151072,359.510928,"LINESTRING (87.69393 23.71135, 87.69458 23.711...",87.724803,23.722809,0.013283,1.413985


In [15]:
latexify(fig_width, fig_height)


In [16]:
fig, ax = plt.subplots(figsize=(10, 5))

waterways.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'blue', linewidth = 0.2, label = 'rivers', alpha=0.8) # river data
data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.2) # UP map

# bk - river distance > 0.5 KM
dist_1 = bk_river_mapping[(bk_river_mapping['distance'] > 0.5)]
ax.scatter(dist_1['lon'],dist_1['lat'], color='green', s=0.5, marker = 'o', linewidth = 0.5, label = f'{len(dist_1)} brick kilns $>$ 0.5 Km')

# bk - river distance < 0.5 KM
dist_1 = bk_river_mapping[bk_river_mapping['distance'] <= 0.5]
ax.scatter(dist_1['lon'],dist_1['lat'], color='red', s=2, marker = 'o',linewidth = 0.5, label = f'{len(dist_1)} brick kilns $<$ 0.5 Km')

# ax.text(76.1, 31.8, 'Himachal\n Pradesh')
ax.text(86.1, 25.73, 'Bihar')
# ax.text(73.8, 26.23, 'Rajasthan')
ax.text(85.9, 24.05, 'Jharkhand')
ax.text(86.9, 23.05, 'West\n Bengal')
# ax.text(75.2, 21.0, 'Maharashtra')
# ax.text(81.2, 21.4, 'Chhattisgarh')

ax.set_xlim(85.73,89.90)
ax.set_ylim(21.29,27.23)
plt.legend( loc='upper left', borderaxespad=0.)

ax.set_axis_off()
format_axes(ax)
# plt.savefig('punjab_bk_river.pdf')
plt.tight_layout()
plt.savefig('west_bengal_bk_river.pdf')
plt.savefig('west_bengal_bk_river.png', dpi=300)

## HIMACHAL PRADESH

In [17]:
bk_river_mapping = cal_bk_river_dist('/home/patel_zeel/compass24/exact_latlon/himachal_pradesh.csv')
bk_river_mapping.head(2)

/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km


,parent,line_id,class,lat,lon,state,x_c,y_c,w,h,...,bottom_right_y,local_top_left_x,local_top_left_y,local_bottom_right_x,local_bottom_right_y,geometry,closest_river_point_long,closest_river_point_lat,dist,distance
0,"31.98,76.78",0,zigzag,31.983763,76.779407,himachal_pradesh,2.393358e+07,1.362802e+07,0.012450,0.012923,...,1.362802e+07,497.79856,139.298488,511.74256,153.771912,"LINESTRING (76.82708 32.02144, 76.81981 32.021...",76.779682,31.988003,0.004249,0.470899
1,"31.38,76.32",0,zigzag,31.376247,76.323428,himachal_pradesh,2.389108e+07,1.369456e+07,0.074682,0.042063,...,1.369458e+07,837.68832,946.166704,921.33216,993.276816,"LINESTRING (76.11541 31.62528, 76.12992 31.602...",76.305027,31.366830,0.020671,2.038292


In [18]:
fig, ax = plt.subplots()

waterways.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'blue', linewidth = 0.2, label = 'rivers', alpha=0.8) # river data
data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.2) # UP map

# bk - river distance > 0.5 KM
dist_1 = bk_river_mapping[(bk_river_mapping['distance'] > 0.5)]
ax.scatter(dist_1['lon'],dist_1['lat'], color='green', s=0.5, marker = 'o', linewidth = 0.5, label = f'{len(dist_1)} brick kilns $>$ 0.5 Km')

# bk - river distance < 0.5 KM
dist_1 = bk_river_mapping[bk_river_mapping['distance'] <= 0.5]
ax.scatter(dist_1['lon'],dist_1['lat'], color='red', s=2, marker = 'o',linewidth = 0.5, label = f'{len(dist_1)} brick kilns $<$ 0.5 Km')

# ax.text(76.1, 31.8, 'Himachal\n Pradesh')
# ax.text(81.1, 26.23, 'Uttar Pradesh')
# ax.text(73.8, 26.23, 'Rajasthan')
# ax.text(77.2, 23.4, 'Madhya Pradesh')
# ax.text(75.2, 21.0, 'Maharashtra')
ax.text(77.5, 33.3, 'Ladakh')
ax.text(76.9, 31.9, 'Himachal\n Pradesh')
ax.text(75.4, 30.9, 'Punjab')
ax.text(77.9, 30.4, 'Uttarakhand')

ax.set_xlim(75.35,79.5)
ax.set_ylim(30.22,33.53)
plt.legend( loc='upper right', borderaxespad=0.)

ax.set_axis_off()
format_axes(ax)
# plt.savefig('punjab_bk_river.pdf')
plt.tight_layout()
plt.savefig('himachal_pradesh_bk_river.pdf')
plt.savefig('himachal_pradesh_bk_river.png', dpi=300)

## MADHYA PRADESH

In [19]:
bk_river_mapping = cal_bk_river_dist('/home/patel_zeel/compass24/exact_latlon/madhya_pradesh.csv')
bk_river_mapping.head(2)

/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km


,parent,line_id,class,lat,lon,state,x_c,y_c,w,h,...,bottom_right_y,local_top_left_x,local_top_left_y,local_bottom_right_x,local_bottom_right_y,geometry,closest_river_point_long,closest_river_point_lat,dist,distance
0,"22.70,77.36",0,zigzag,22.702487,77.364818,madhya_pradesh,2.398814e+07,1.460354e+07,0.030983,0.026679,...,1.460356e+07,991.686416,293.787928,1026.387824,323.668072,"LINESTRING (77.10336 23.04459, 77.10370 23.044...",77.363724,22.702596,0.001099,0.113056
1,"26.62,77.94",0,fcbk,26.620586,77.941580,madhya_pradesh,2.404190e+07,1.420159e+07,0.028714,0.064569,...,1.420162e+07,691.181176,462.726320,723.340744,535.043600,"LINESTRING (77.31030 26.30318, 77.31296 26.306...",77.944065,26.658208,0.037704,4.175788


In [20]:
latexify(fig_width, fig_height)


In [21]:
fig, ax = plt.subplots()

waterways.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'blue', linewidth = 0.2, label = 'rivers', alpha=0.8) # river data
data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.2) # UP map

# bk - river distance > 0.5 KM
dist_1 = bk_river_mapping[(bk_river_mapping['distance'] > 0.5)]
ax.scatter(dist_1['lon'],dist_1['lat'], color='green', s=0.5, marker = 'o', linewidth = 0.5, label = f'{len(dist_1)} brick kilns $>$ 0.5 Km')

# bk - river distance < 0.5 KM
dist_1 = bk_river_mapping[bk_river_mapping['distance'] <= 0.5]
ax.scatter(dist_1['lon'],dist_1['lat'], color='red', s=2, marker = 'o',linewidth = 0.5, label = f'{len(dist_1)} brick kilns $<$ 0.5 Km')

# ax.text(76.1, 31.8, 'Himachal\n Pradesh')
ax.text(81.1, 26.23, 'Uttar Pradesh')
ax.text(73.8, 26.23, 'Rajasthan')
ax.text(77.2, 23.4, 'Madhya Pradesh')
ax.text(75.2, 21.0, 'Maharashtra')
ax.text(81.2, 21.4, 'Chhattisgarh')

ax.set_xlim(73.5,83.5)
ax.set_ylim(20.65,27.23)
plt.legend( loc='upper right', borderaxespad=0.)

ax.set_axis_off()
format_axes(ax)
# plt.savefig('punjab_bk_river.pdf')
plt.tight_layout()
plt.savefig('madhya_pradesh_bk_river.pdf')
plt.savefig('madhya_pradesh_bk_river.png', dpi=300)

## UTTAR PRADESH

In [22]:
bk_river_mapping = cal_bk_river_dist('/home/patel_zeel/compass24/exact_latlon/uttar_pradesh.csv')
bk_river_mapping.head(2)

/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km


,parent,line_id,class,lat,lon,state,x_c,y_c,w,h,...,bottom_right_y,local_top_left_x,local_top_left_y,local_bottom_right_x,local_bottom_right_y,geometry,closest_river_point_long,closest_river_point_lat,dist,distance
0,"27.72,80.91",0,fcbk,27.716037,80.905317,uttar_pradesh,2.431814e+07,1.408682e+07,0.074447,0.060359,...,1.408685e+07,81.824288,943.44068,165.205152,1011.04220,"LINESTRING (80.96631 28.06533, 80.96334 28.063...",80.918817,27.712030,0.014082,1.403440
1,"27.72,80.91",1,fcbk,27.719278,80.908307,uttar_pradesh,2.431842e+07,1.408648e+07,0.067373,0.057251,...,1.408651e+07,364.450912,603.93732,439.908448,668.05788,"LINESTRING (80.96631 28.06533, 80.96334 28.063...",80.919782,27.714935,0.012269,1.229704


In [23]:
print('min lat:', bk_river_mapping['lat'].min())
print('max lat:', bk_river_mapping['lat'].max())
print('min lon:', bk_river_mapping['lon'].min())
print('max lon:', bk_river_mapping['lon'].max())

min lat: 22.222647842157727
max lat: 30.149494836330074
min lon: 77.12507550304413
max lon: 86.79417717704774


In [24]:
latexify(fig_width, fig_height)


In [25]:
fig, ax = plt.subplots(figsize=(10, 5))

waterways.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'blue', linewidth = 0.2, label = 'rivers', alpha=0.8) # river data
data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.2) # UP map

# bk - river distance > 0.5 KM
dist_1 = bk_river_mapping[(bk_river_mapping['distance'] > 0.5)]
ax.scatter(dist_1['lon'],dist_1['lat'], color='green', s=0.5, marker = 'o', linewidth = 0.5, label = f'{len(dist_1)} brick kilns $>$ 0.5 Km')

# bk - river distance < 0.5 KM
dist_1 = bk_river_mapping[bk_river_mapping['distance'] <= 0.5]
ax.scatter(dist_1['lon'],dist_1['lat'], color='red', s=2, marker = 'o',linewidth = 0.5, label = f'{len(dist_1)} brick kilns $<$ 0.5 Km')

# ax.text(76.1, 31.8, 'Himachal\n Pradesh')
ax.text(77.9, 30.2, 'Uttarakhand')
ax.text(78.7, 27.7, 'Uttar Pradesh')
# ax.text(74, 30.2, 'Rajasthan')
ax.text(77.2, 23.4, 'Madhya Pradesh')

ax.set_xlim(77,87)
ax.set_ylim(22,31)
plt.legend( loc='upper right', borderaxespad=0.)

ax.set_axis_off()
format_axes(ax)
# plt.savefig('punjab_bk_river.pdf')
plt.tight_layout()
plt.savefig('uttar_pradesh_bk_river.pdf')
plt.savefig('uttar_pradesh_bk_river.png', dpi=300)

# PUNJAB

In [26]:
bk_river_mapping = cal_bk_river_dist('/home/patel_zeel/compass24/exact_latlon/punjab.csv')
bk_river_mapping.head(2)

/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km


,parent,line_id,class,lat,lon,state,x_c,y_c,w,h,...,bottom_right_y,local_top_left_x,local_top_left_y,local_bottom_right_x,local_bottom_right_y,geometry,closest_river_point_long,closest_river_point_lat,dist,distance
0,"30.79,75.87",0,zigzag,30.788065,75.871358,punjab,2.384894e+07,1.375857e+07,0.115034,0.045408,...,1.375859e+07,622.157760,744.50432,750.995840,795.36128,"LINESTRING (76.01536 30.78311, 76.00704 30.782...",75.871653,30.77777,0.010300,1.14178
1,"30.79,75.87",1,zigzag,30.787651,75.875818,punjab,2.384935e+07,1.375861e+07,0.031660,0.100504,...,1.375867e+07,1084.514928,758.56144,1119.974352,871.12592,"LINESTRING (76.01536 30.78311, 76.00704 30.782...",75.879447,30.77943,0.008986,0.97535


textwidth & columnwidth & linewidth are 506.295pt

columnwidth & linewidth are 241.14749pt


In [27]:
latexify(fig_width, fig_height)

In [28]:
fig, ax = plt.subplots(figsize=(10, 5))

waterways.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'blue', linewidth = 0.2, label = 'rivers', alpha=0.8) # river data
data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Punjab map

# bk - river distance > 0.5 KM
dist_1 = bk_river_mapping[(bk_river_mapping['distance'] > 0.5)]
ax.scatter(dist_1['lon'],dist_1['lat'], color='green', s=2, marker = 'o', linewidth = 0.5, label = f'{len(dist_1)} brick kilns $>$ 0.5 Km')

# bk - river distance < 0.5 KM
dist_1 = bk_river_mapping[bk_river_mapping['distance'] <= 0.5]
ax.scatter(dist_1['lon'],dist_1['lat'], color='red', s=2, marker = 'o',linewidth = 0.5, label = f'{len(dist_1)} brick kilns $<$ 0.5 Km')

ax.text(76.1, 31.8, 'Himachal\n Pradesh')
ax.text(75.4, 30.9, 'Punjab')
ax.text(74, 29.6, 'Rajasthan')
ax.text(76, 29.6, 'Haryana')

ax.set_xlim(73.5,77)
ax.set_ylim(29.5,32.5)
plt.legend( loc='upper left', borderaxespad=0.)

ax.set_axis_off()
format_axes(ax)
# plt.savefig('punjab_bk_river.pdf')
plt.tight_layout()
plt.savefig('punjab_bk_river.pdf')
plt.savefig('punjab_bk_river.png', dpi=300)

# HARYANA

In [29]:
bk_river_mapping = cal_bk_river_dist('/home/patel_zeel/compass24/exact_latlon/haryana.csv')

/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km


In [30]:
fig, ax = plt.subplots()

waterways.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'blue', linewidth = 0.2, label = 'rivers', alpha=0.8) # river data
data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Haryana map


# bk - river distance > 0.5 KM
dist_1 = bk_river_mapping[(bk_river_mapping['distance'] > 0.5)]
ax.scatter(dist_1['lon'],dist_1['lat'], color='green', s=2, marker = 'o', linewidth = 0.5, label = f'{len(dist_1)} brick kilns $>$ 0.5 Km')

# bk - river distance < 0.5 KM
dist_1 = bk_river_mapping[bk_river_mapping['distance'] <= 0.5]
ax.scatter(dist_1['lon'],dist_1['lat'], color='red', s=2, marker = 'o',linewidth = 0.5, label = f'{len(dist_1)} brick kilns $<$ 0.5 Km')

ax.text(77.3, 29.5, 'Uttar\n Pradesh')
ax.text(74.5, 28.5, 'Rajasthan')
ax.text(75.5, 30.5, 'Punjab')
ax.text(75.8, 28.5, 'Haryana')
ax.text(77, 28.6, 'New Delhi')

ax.set_xlim(74,78)
ax.set_ylim(27.5,31)
plt.legend(loc = "upper left", borderaxespad=0.)
format_axes(ax)
ax.set_axis_off()
plt.tight_layout()
plt.savefig('haryana_bk_river.pdf')
plt.savefig('haryana_bk_river.png', dpi=300)


# BIHAR

In [31]:
bk_river_mapping = cal_bk_river_dist('/home/patel_zeel/compass24/exact_latlon/bihar.csv')

/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km


In [32]:
fig, ax = plt.subplots()

waterways.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'blue', linewidth = 0.3, label = 'rivers') # river data
data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Bihar map


# bk - river distance > 0.5 KM
dist_1 = bk_river_mapping[(bk_river_mapping['distance'] > 0.5)]
ax.scatter(dist_1['lon'],dist_1['lat'], color='green', s=2, marker = 'o', linewidth = 0.5, label = f'{len(dist_1)} brick kilns $>$ 0.5 Km')

# bk - river distance < 0.5 KM
dist_1 = bk_river_mapping[bk_river_mapping['distance'] <= 0.5]
ax.scatter(dist_1['lon'],dist_1['lat'], color='red', s=2, marker = 'o',linewidth = 0.5, label = f'{len(dist_1)} brick kilns $<$ 0.5 Km')

ax.text(83.2, 25.8, 'Uttar Pradesh')
ax.text(85.6, 24.3, 'Jharkhand')
ax.text(86, 26.3, 'Bihar' )
ax.text(88, 25.3, 'West Bengal')

ax.set_xlim(83,89)
ax.set_ylim(24.25,27.6)
plt.legend(loc = 'upper center')
format_axes(ax)
ax.set_axis_off()
# plt.savefig('bihar_bk_river.pdf')
# plt.savefig('bihar_bk_river.png')

# Uttarakhand

In [33]:
bk_river_mapping = cal_bk_river_dist('/home/patel_zeel/compass24/exact_latlon/uttarakhand.csv')

/tmp/ipykernel_2136890/1332830719.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_river_mapping['distance'][i] = geopy.distance.distance((bk_river_mapping['lat'][i],bk_river_mapping['lon'][i]), (bk_river_mapping['closest_river_point_lat'][i],bk_river_mapping['closest_river_point_long'][i])).km


In [34]:
# min and max value of lat and lon
print('min lat:', bk_river_mapping['lat'].min())
print('max lat:', bk_river_mapping['lat'].max())
print('min lon:', bk_river_mapping['lon'].min())
print('max lon:', bk_river_mapping['lon'].max())

min lat: 28.79500167631892
max lat: 30.75664022234773
min lon: 77.58381075210572
max lon: 80.38575352287293


In [35]:
fig, ax = plt.subplots()

waterways.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'blue', linewidth = 0.3, label = 'rivers') # river data
data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Bihar map


# bk - river distance > 0.5 KM
dist_1 = bk_river_mapping[(bk_river_mapping['distance'] > 0.5)]
ax.scatter(dist_1['lon'],dist_1['lat'], color='green', s=2, marker = 'o', linewidth = 0.5, label = f'{len(dist_1)} brick kilns $>$ 0.5 Km')

# bk - river distance < 0.5 KM
dist_1 = bk_river_mapping[bk_river_mapping['distance'] <= 0.5]
ax.scatter(dist_1['lon'],dist_1['lat'], color='red', s=2, marker = 'o',linewidth = 0.5, label = f'{len(dist_1)} brick kilns $<$ 0.5 Km')

# ax.text(83.2, 25.8, 'Uttar Pradesh')
# ax.text(76.1, 31.8, 'Himachal\n Pradesh')
ax.text(78.4, 30.2, 'Uttarakhand')
# ax.text(85.6, 24.3, 'Jharkhand')
# ax.text(86, 26.3, 'Bihar' )
# ax.text(88, 25.3, 'West Bengal')

ax.set_xlim(77.5,80.5)
ax.set_ylim(28.77,30.77)
plt.legend(loc = 'upper right')
format_axes(ax)
ax.set_axis_off()
# plt.savefig('bihar_bk_river.pdf')
# plt.savefig('bihar_bk_river.png')

BRICK KILN HOSPITAL DISTANCE

In [36]:
hospital_df = pd.read_csv('/home/rishabh.mondal/bkdb/India_Hospital_Data.csv')
hospital_df = hospital_df.rename(columns = {'lon' : 'lon', 'lat' : 'lat'})

In [37]:
def ckdnearest_hospital(brick_kilns, hospital_df):
    A = np.vstack([np.array(geom) for geom in brick_kilns[['lon','lat']].values])
    B = np.vstack([np.array(geom) for geom in hospital_df[['lon','lat']].values] )
    ckd_tree = cKDTree(B)
    dist, hospital_idx = ckd_tree.query(A, k=1)
    closest_hospital_point = B[hospital_idx]
    gdf = pd.concat(
        [brick_kilns,
         pd.DataFrame(closest_hospital_point, columns = ['closest_hospital_long','closest_hospital_lat']),
         pd.Series(dist, name='dist')], axis=1)
    return gdf

PUNJAB

In [39]:
bk_hospital_mapping = cal_bk_hosp_dist('/home/patel_zeel/compass24/exact_latlon/punjab.csv')
bk_hospital_mapping.head(2)

/tmp/ipykernel_2136890/1286038133.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km


,parent,line_id,class,lat,lon,state,x_c,y_c,w,h,...,bottom_right_x,bottom_right_y,local_top_left_x,local_top_left_y,local_bottom_right_x,local_bottom_right_y,closest_hospital_long,closest_hospital_lat,dist,distance_km
0,"30.79,75.87",0,zigzag,30.788065,75.871358,punjab,2.384894e+07,1.375857e+07,0.115034,0.045408,...,2.384900e+07,1.375859e+07,622.157760,744.50432,750.995840,795.36128,75.883065,30.801756,0.018013,1.886587
1,"30.79,75.87",1,zigzag,30.787651,75.875818,punjab,2.384935e+07,1.375861e+07,0.031660,0.100504,...,2.384937e+07,1.375867e+07,1084.514928,758.56144,1119.974352,871.12592,75.883065,30.801756,0.015858,1.710660


In [40]:
print('min lat:', bk_hospital_mapping['lat'].min())
print('max lat:', bk_hospital_mapping['lat'].max())
print('min lon:', bk_hospital_mapping['lon'].min())
print('max lon:', bk_hospital_mapping['lon'].max())

min lat: 29.60109779771108
max lat: 32.42025663897406
min lon: 73.9561200460434
max lon: 76.91218372154238


In [41]:
fig, ax = plt.subplots(figsize=(8, 4))

vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] <= 0.8]
non_vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] > 0.8]
data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Punjab map
ax.scatter(bk_hospital_mapping['closest_hospital_long'], bk_hospital_mapping['closest_hospital_lat'],color = 'blue', marker='+', label = f'hospitals', alpha=0.5) # hospital data
# ax.scatter(non_vulnerable_hospital['closest_hospital_long'], non_vulnerable_hospital['closest_hospital_lat'],color = 'blue', marker='+', label = f'{len(non_vulnerable_hospital)} hospitals $>$ 0.8 Km') # hospital data

ax.scatter(vulnerable_hospital['lon'], vulnerable_hospital['lat'], color = 'red', s=2, label = f'{len(vulnerable_hospital)} brick kilns $<$ 0.8 Km') # hospital data
ax.scatter(non_vulnerable_hospital['lon'], non_vulnerable_hospital['lat'], color = 'green', s=0.5, label = f'{len(non_vulnerable_hospital)} brick kilns $>$ 0.8 Km') # hospital data

ax.text(76, 32, 'Himachal\n Pradesh')
ax.text(75.5, 31, 'Punjab')
ax.text(74, 29.6, 'Rajasthan')
ax.text(76, 29.6, 'Haryana')

ax.set_xlim(73,77)
ax.set_ylim(29.5,32.5)
plt.legend(loc='upper left')
ax.set_axis_off()
plt.tight_layout()
plt.savefig('punjab_bk_hosp.pdf')
plt.savefig('punjab_bk_hosp.png', dpi=300)

HARYANA

In [42]:
bk_hospital_mapping = cal_bk_hosp_dist('/home/patel_zeel/compass24/exact_latlon/haryana.csv')

vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] <= 0.8]
non_vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] > 0.8]

fig, ax = plt.subplots()

data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Haryana map
ax.scatter(bk_hospital_mapping['closest_hospital_long'], bk_hospital_mapping['closest_hospital_lat'],color = 'blue', marker='+', label = f'hospitals', alpha=0.5) # hospital data

ax.scatter(vulnerable_hospital['lon'], vulnerable_hospital['lat'], color = 'red', s=2, label = f'{len(vulnerable_hospital)} brick kilns $<$ 0.8 Km') # hospital data
ax.scatter(non_vulnerable_hospital['lon'], non_vulnerable_hospital['lat'], color = 'green', s=2, label = f'{len(non_vulnerable_hospital)} brick kilns $>$ 0.8 Km') # hospital data

ax.text(77.3, 29.5, 'Uttar \nPradesh')
ax.text(74.5, 28.5, 'Rajasthan')
ax.text(75.5, 30.5, 'Punjab')
ax.text(76, 29, 'Haryana')
ax.text(77, 28.6, 'New Delhi')

ax.set_xlim(74,78)
ax.set_ylim(27.5,31)
plt.legend()
ax.set_axis_off()
plt.tight_layout()
plt.savefig('haryana_bk_hosp.pdf')
plt.savefig('haryana_bk_hosp.png', dpi=300)

/tmp/ipykernel_2136890/1286038133.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km


Bihar

In [43]:
vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] <= 0.8]
vulnerable_hospital

,parent,line_id,class,lat,lon,state,x_c,y_c,w,h,...,bottom_right_x,bottom_right_y,local_top_left_x,local_top_left_y,local_bottom_right_x,local_bottom_right_y,closest_hospital_long,closest_hospital_lat,dist,distance_km
1,"28.93,76.87",1,zigzag,28.925854,76.874942,haryana,2.394248e+07,1.395872e+07,0.106513,0.076282,...,2.394254e+07,1.395876e+07,960.954960,958.812960,1080.249520,1044.248800,76.880000,28.930000,0.006540,0.674110
5,"29.51,76.68",0,zigzag,29.512676,76.679719,haryana,2.392428e+07,1.389605e+07,0.040305,0.088932,...,2.392431e+07,1.389610e+07,511.279328,223.557824,556.421152,323.161216,76.684581,29.508006,0.006741,0.700122
6,"29.51,76.68",1,zigzag,29.507501,76.680411,haryana,2.392435e+07,1.389660e+07,0.083950,0.035142,...,2.392440e+07,1.389662e+07,551.304096,807.935408,645.328544,847.294672,76.684581,29.508006,0.004200,0.408163
55,"28.61,76.80",0,zigzag,28.614335,76.801009,haryana,2.393559e+07,1.399184e+07,0.035000,0.071490,...,2.393561e+07,1.399188e+07,634.423776,59.697176,673.624224,139.765640,76.800000,28.610000,0.004451,0.490482
56,"28.61,76.80",1,zigzag,28.613355,76.803794,haryana,2.393585e+07,1.399195e+07,0.056106,0.086995,...,2.393588e+07,1.399200e+07,882.223104,155.078056,945.061376,252.512344,76.800000,28.610000,0.005065,0.525312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2243,"29.82,76.96",0,zigzag,29.818768,76.961193,haryana,2.395052e+07,1.386321e+07,0.096434,0.031216,...,2.395057e+07,1.386323e+07,617.234296,674.880752,725.240264,709.842448,76.956385,29.816375,0.005371,0.535159
2281,"28.21,76.69",0,zigzag,28.207585,76.688920,haryana,2.392514e+07,1.403495e+07,0.077586,0.042316,...,2.392518e+07,1.403497e+07,415.912168,791.688856,502.808152,839.082664,76.684140,28.205787,0.005107,0.509863
2338,"29.21,76.24",1,zigzag,29.211255,76.238666,haryana,2.388317e+07,1.392828e+07,0.094878,0.032542,...,2.388323e+07,1.392830e+07,382.551904,407.709064,488.814816,444.156216,76.245229,29.212100,0.006617,0.644991
2339,"29.21,76.24",2,zigzag,29.208665,76.239248,haryana,2.388323e+07,1.392856e+07,0.097078,0.041412,...,2.388328e+07,1.392858e+07,435.563296,679.369376,544.291104,725.751264,76.245229,29.212100,0.006897,0.695121


In [44]:
bk_hospital_mapping = cal_bk_hosp_dist('/home/patel_zeel/compass24/exact_latlon/bihar.csv')

vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] <= 0.8]
non_vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] > 0.8]


fig, ax = plt.subplots()

data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Bihar map
ax.scatter(vulnerable_hospital['closest_hospital_long'], vulnerable_hospital['closest_hospital_lat'],color = 'red', marker='+', label = f'{len(vulnerable_hospital)} hospitals $<=$ 0.88 Km') # hospital data
ax.scatter(non_vulnerable_hospital['closest_hospital_long'], non_vulnerable_hospital['closest_hospital_lat'],color = 'blue', marker='+', label = f'{len(non_vulnerable_hospital)} hospitals $>$ 0.88 Km') # hospital data
ax.scatter(vulnerable_hospital['lon'], vulnerable_hospital['lat'],color = 'black', marker='.', label = 'Brick kilns') # brick kilns

ax.text(83.2, 25.8, 'Uttar\n Pradesh')
ax.text(85.6, 24.3, 'Jharkhand')
ax.text(86, 26.3, 'Bihar' )
ax.text(88, 25.3, 'West\n Bengal')

ax.set_xlim(83,89)
ax.set_ylim(24.25,27.6)
plt.legend(loc = 'upper center')
ax.set_axis_off()
# plt.savefig('bihar_bk_hosp.pdf')
# plt.savefig('bihar_bk_hosp.png')

/tmp/ipykernel_2136890/1286038133.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km


JHARKHAND

In [45]:
bk_hospital_mapping = cal_bk_hosp_dist('/home/patel_zeel/compass24/exact_latlon/jharkhand.csv')

vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] <= 0.8]
non_vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] > 0.8]


fig, ax = plt.subplots(figsize=(8, 4))

data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Jharkhand map
ax.scatter(vulnerable_hospital['closest_hospital_long'], vulnerable_hospital['closest_hospital_lat'],color = 'red', marker='+', label = f'{len(vulnerable_hospital)} hospitals $<=$ 0.88 Km') # hospital data
ax.scatter(non_vulnerable_hospital['closest_hospital_long'], non_vulnerable_hospital['closest_hospital_lat'],color = 'blue', marker='+', label = f'{len(non_vulnerable_hospital)} hospitals $>$ 0.88 Km') # hospital data
ax.scatter(vulnerable_hospital['lon'], vulnerable_hospital['lat'],color = 'black', marker='.', label = 'Brick kilns') # brick kilns

ax.text(85.1, 24.93, 'Bihar')
# ax.text(73.8, 26.23, 'Rajasthan')
ax.text(84.6, 23.45, 'Jharkhand')
ax.text(86.9, 23.05, 'West\n Bengal')
# ax.text(75.2, 21.0, 'Maharashtra')
ax.text(83.05, 22.9, 'Chhattisgarh')

ax.set_xlim(82.97,88.00)
ax.set_ylim(21.87,25.63)
plt.legend(loc = 'upper center')
ax.set_axis_off()
plt.savefig('jharkhand_bk_hosp.pdf')
plt.savefig('jharkhand_bk_hosp.png')

/tmp/ipykernel_2136890/1286038133.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km


WEST BENGAL

In [46]:
bk_hospital_mapping = cal_bk_hosp_dist('/home/patel_zeel/compass24/exact_latlon/west_bengal.csv')

vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] <= 0.8]
non_vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] > 0.8]


fig, ax = plt.subplots(figsize=(11, 7))

data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Jharkhand map
ax.scatter(vulnerable_hospital['closest_hospital_long'], vulnerable_hospital['closest_hospital_lat'],color = 'red', marker='+', label = f'{len(vulnerable_hospital)} hospitals $<=$ 0.88 Km') # hospital data
ax.scatter(non_vulnerable_hospital['closest_hospital_long'], non_vulnerable_hospital['closest_hospital_lat'],color = 'blue', marker='+', label = f'{len(non_vulnerable_hospital)} hospitals $>$ 0.88 Km') # hospital data
ax.scatter(vulnerable_hospital['lon'], vulnerable_hospital['lat'],color = 'black', marker='.', label = 'Brick kilns') # brick kilns

ax.text(86.1, 25.73, 'Bihar')
# ax.text(73.8, 26.23, 'Rajasthan')
ax.text(85.9, 24.05, 'Jharkhand')
ax.text(86.9, 23.05, 'West\n Bengal')
# ax.text(75.2, 21.0, 'Maharashtra')
# ax.text(81.2, 21.4, 'Chhattisgarh')

ax.set_xlim(85.73,89.90)
ax.set_ylim(21.29,27.23)
plt.legend(loc = 'upper left')
ax.set_axis_off()
plt.savefig('west_bengal_bk_hosp.pdf')
plt.savefig('west_bengal_bk_hosp.png')

/tmp/ipykernel_2136890/1286038133.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km


HIMACHAL PRADESH

In [47]:
bk_hospital_mapping = cal_bk_hosp_dist('/home/patel_zeel/compass24/exact_latlon/himachal_pradesh.csv')

vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] <= 0.8]
non_vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] > 0.8]


fig, ax = plt.subplots(figsize=(8, 4))

data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Jharkhand map
ax.scatter(vulnerable_hospital['closest_hospital_long'], vulnerable_hospital['closest_hospital_lat'],color = 'red', marker='+', label = f'{len(vulnerable_hospital)} hospitals $<=$ 0.88 Km') # hospital data
ax.scatter(non_vulnerable_hospital['closest_hospital_long'], non_vulnerable_hospital['closest_hospital_lat'],color = 'blue', marker='+', label = f'{len(non_vulnerable_hospital)} hospitals $>$ 0.88 Km') # hospital data
ax.scatter(vulnerable_hospital['lon'], vulnerable_hospital['lat'],color = 'black', marker='.', label = 'Brick kilns') # brick kilns

# ax.text(81.1, 26.23, 'Uttar Pradesh')
# ax.text(73.8, 26.23, 'Rajasthan')
# ax.text(77.2, 23.4, 'Madhya Pradesh')
# ax.text(75.2, 21.0, 'Maharashtra')
ax.text(77.5, 33.3, 'Ladakh')
ax.text(76.9, 31.9, 'Himachal\n Pradesh')
ax.text(75.4, 30.9, 'Punjab')
ax.text(77.9, 30.4, 'Uttarakhand')

ax.set_xlim(75.35,79.5)
ax.set_ylim(30.22,33.53)

plt.legend(loc = 'upper right')
ax.set_axis_off()
plt.savefig('himachal_pradesh_bk_hosp.pdf')
plt.savefig('himachal_pradesh_bk_hosp.png')

/tmp/ipykernel_2136890/1286038133.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km


MADHYA PRADESH

In [48]:
bk_hospital_mapping = cal_bk_hosp_dist('/home/patel_zeel/compass24/exact_latlon/madhya_pradesh.csv')

vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] <= 0.8]
non_vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] > 0.8]


fig, ax = plt.subplots(figsize=(8, 4))

data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Jharkhand map
ax.scatter(vulnerable_hospital['closest_hospital_long'], vulnerable_hospital['closest_hospital_lat'],color = 'red', marker='+', label = f'{len(vulnerable_hospital)} hospitals $<=$ 0.88 Km') # hospital data
ax.scatter(non_vulnerable_hospital['closest_hospital_long'], non_vulnerable_hospital['closest_hospital_lat'],color = 'blue', marker='+', label = f'{len(non_vulnerable_hospital)} hospitals $>$ 0.88 Km') # hospital data
ax.scatter(vulnerable_hospital['lon'], vulnerable_hospital['lat'],color = 'black', marker='.', label = 'Brick kilns') # brick kilns

ax.text(81.1, 26.23, 'Uttar Pradesh')
ax.text(73.8, 26.23, 'Rajasthan')
ax.text(77.2, 23.4, 'Madhya Pradesh')
ax.text(75.2, 21.0, 'Maharashtra')
ax.text(81.2, 21.4, 'Chhattisgarh')

ax.set_xlim(73.5,83.5)
ax.set_ylim(20.65,27.23)

plt.legend(loc = 'upper right')
ax.set_axis_off()
plt.savefig('madhya_pradesh_bk_hosp.pdf')
plt.savefig('madhya_pradesh_bk_hosp.png')

/tmp/ipykernel_2136890/1286038133.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km


UTTAR PRADESH

In [49]:
bk_hospital_mapping = cal_bk_hosp_dist('/home/patel_zeel/compass24/exact_latlon/uttar_pradesh.csv')

vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] <= 0.8]
non_vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] > 0.8]


fig, ax = plt.subplots(figsize = (8,4))

data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Jharkhand map
ax.scatter(vulnerable_hospital['closest_hospital_long'], vulnerable_hospital['closest_hospital_lat'],color = 'red', marker='+', label = f'{len(vulnerable_hospital)} hospitals $<=$ 0.88 Km') # hospital data
ax.scatter(non_vulnerable_hospital['closest_hospital_long'], non_vulnerable_hospital['closest_hospital_lat'],color = 'blue', marker='+', label = f'{len(non_vulnerable_hospital)} hospitals $>$ 0.88 Km') # hospital data
ax.scatter(vulnerable_hospital['lon'], vulnerable_hospital['lat'],color = 'black', marker='.', label = 'Brick kilns') # brick kilns

ax.text(77.9, 30.2, 'Uttarakhand')
ax.text(78.7, 27.7, 'Uttar Pradesh')
# ax.text(74, 30.2, 'Rajasthan')
ax.text(77.2, 23.4, 'Madhya Pradesh')

ax.set_xlim(76.90,84.90)
ax.set_ylim(23.2,31.38)

plt.legend(loc = 'upper right')
ax.set_axis_off()
plt.savefig('uttar_pradesh_bk_hosp.pdf')
plt.savefig('uttar_pradesh_bk_hosp.png')

/tmp/ipykernel_2136890/1286038133.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km


### Uttarakhand

In [50]:
bk_hospital_mapping = cal_bk_hosp_dist('/home/patel_zeel/compass24/exact_latlon/uttarakhand.csv')

vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] <= 0.8]
non_vulnerable_hospital = bk_hospital_mapping[bk_hospital_mapping['distance_km'] > 0.8]


fig, ax = plt.subplots(figsize = (8,4))

data2.plot(ax=ax, cmap = 'Pastel2', edgecolor = 'black', linewidth = 0.1) # Jharkhand map
ax.scatter(vulnerable_hospital['closest_hospital_long'], vulnerable_hospital['closest_hospital_lat'],color = 'red', marker='+', label = f'{len(vulnerable_hospital)} hospitals $<=$ 0.88 Km') # hospital data
ax.scatter(non_vulnerable_hospital['closest_hospital_long'], non_vulnerable_hospital['closest_hospital_lat'],color = 'blue', marker='+', label = f'{len(non_vulnerable_hospital)} hospitals $>$ 0.88 Km') # hospital data
ax.scatter(vulnerable_hospital['lon'], vulnerable_hospital['lat'],color = 'black', marker='.', label = 'Brick kilns') # brick kilns

ax.text(77.9, 30.2, 'Uttarakhand')
# ax.text(78.7, 27.7, 'Uttar Pradesh')
# ax.text(74, 30.2, 'Rajasthan')
# ax.text(77.2, 23.4, 'Madhya Pradesh')

ax.set_xlim(77.5,80.5)
ax.set_ylim(28.77,30.77)

plt.legend(loc = 'lower left')
ax.set_axis_off()
plt.savefig('uttarakhand_bk_hosp.pdf')
plt.savefig('uttarakhand_bk_hosp.png')

/tmp/ipykernel_2136890/1286038133.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance((bk_hospital_mapping['lat'][i],bk_hospital_mapping['lon'][i]), (bk_hospital_mapping['closest_hospital_lat'][i],bk_hospital_mapping['closest_hospital_long'][i])).km


In [51]:
vulnerable_hospital_hp

NameError: name 'vulnerable_hospital_hp' is not defined

In [52]:
vulnerable_hospital_mp

NameError: name 'vulnerable_hospital_mp' is not defined

In [53]:
vulnerable_hospital_up

NameError: name 'vulnerable_hospital_up' is not defined

In [54]:
df = pd.concat([vulnerable_hospital_hp, vulnerable_hospital_mp, vulnerable_hospital_up, vulnerable_hospital_b, vulnerable_hospital_j, vulnerable_hospital_w, vulnerable_hospital_ut, vulnerable_hospital_h,vulnerable_hospital_p], axis = 0)
df

NameError: name 'vulnerable_hospital_hp' is not defined

In [55]:
df1 = pd.concat([dist_hp, dist_mp, dist_up, dist_b, dist_j, dist_w, dist_ut, dist_h, dist_p], axis = 0)
df1

NameError: name 'dist_hp' is not defined

In [56]:
# find common brick kilns in both vulnerable to river and hospital
common = pd.merge(df, df1, how='inner', on=['lat', 'lon'])
common

NameError: name 'df' is not defined